In [5]:
import pandas as pd
import numpy as np

class Perceptron:
    def __init__(self, num_features, learning_rate=0.01):
        self.weights = np.random.rand(num_features)
        self.bias = np.random.rand()
        self.learning_rate = learning_rate

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def predict(self, inputs):
        weighted_sum = np.dot(inputs, self.weights) + self.bias
        return self.sigmoid(weighted_sum)

    def train(self, inputs, labels, epochs):
        for _ in range(epochs):
            for x, y in zip(inputs, labels):
                prediction = self.predict(x)
                error = y - prediction
                self.weights += self.learning_rate * error * x
                self.bias += self.learning_rate * error

# Load data from Excel sheet into DataFrame
df = pd.read_excel('customerdata.xlsx')

print(df)

# Encoding labels to numeric values
label_encoding = {'Yes': 1, 'No': 0}
df['High Value Tx'] = df['High Value Tx'].map(label_encoding)

# Extracting features and labels
inputs = df.drop(columns=['Customer', 'High Value Tx']).values.astype(float)
labels = df['High Value Tx'].values

# Normalize inputs
inputs = inputs / inputs.max(axis=0)

# Split data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs, labels, test_size=0.2, random_state=42)

# Initialize and train the perceptron
perceptron = Perceptron(num_features=X_train.shape[1])
perceptron.train(X_train, y_train, epochs=1000)

# Test the perceptron on test data
correct_predictions_perceptron = 0
for x, y in zip(X_test, y_test):
    prediction = perceptron.predict(x)
    if prediction >= 0.5:
        predicted_class = 1
    else:
        predicted_class = 0
    if predicted_class == y:
        correct_predictions_perceptron += 1

accuracy_perceptron = correct_predictions_perceptron / len(y_test)
print(f"Accuracy of perceptron on test data: {accuracy_perceptron}")

# Using matrix pseudo-inverse
X_train_pseudo_inv = np.linalg.pinv(X_train)
weights_pseudo_inv = np.dot(X_train_pseudo_inv, y_train)

# Test the pseudo-inverse on test data
predictions_pseudo_inv = np.dot(X_test, weights_pseudo_inv)
predictions_pseudo_inv = np.where(predictions_pseudo_inv >= 0.5, 1, 0)

accuracy_pseudo_inv = np.mean(predictions_pseudo_inv == y_test)
print(f"Accuracy of matrix pseudo-inverse on test data: {accuracy_pseudo_inv}")


Accuracy of perceptron on test data: 0.5
Accuracy of matrix pseudo-inverse on test data: 0.5
